In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import pylab
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm



In [ ]:
startup=pd.read_csv("/content/50_Startups.csv")


In [ ]:
startup.head()

In [ ]:
#converting categorical i/p to numerical
dummy_states=pd.get_dummies(startup['State'])
# renaming dummy
dummy_states.columns=["State"+str(i) for i in range(0, 3)]
dummy_states.head()


In [ ]:
#droping State from startup and cocatinating  dummy as well as
startup=startup.drop(['State'], axis=1)
frames=[dummy_states,startup]
final_startup=pd.concat(frames,axis=1)
final_startup.head()

In [ ]:
#checking correlation for feature selection
corr1=pd.DataFrame(final_startup.iloc[:,:-1].corr())
corr1

In [ ]:
#feature selection checking feature which exceeds threshold value
threshold=0.75
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

correlation(final_startup.iloc[:,:-1],threshold)
#none of feature exceeding our threshold value


In [ ]:
#renaming all features
final_startup.columns='state0','state1','state2','rd','admin','market','profit'
final_startup.head()

In [ ]:
#model 1

model1=smf.ols('profit~state0+state1+state2+rd+admin+market',data=final_startup).fit()
model1.summary()

#here R squared value is >0.80  strong model
#but  adminand market  p value is >0.05 but we cannot remove those fetures as its playing important role 

In [ ]:
#check for influencial value
sm.graphics.influence_plot(model1)
#here 49 is looking most influenceable value 

In [ ]:
#dropiing most influencer row
final_startup=final_startup.drop(final_startup.index[49])

In [ ]:
#model 2
#transformation feature

model2=smf.ols('profit~state0+state1+state2+rd+np.log(admin)+market',data=final_startup).fit()
model2.summary()

#here R squared value is >0.80  strong model
#but  admin and market  p value is >0.05 but we cannot remove those fetures as its playing important role 

In [ ]:
#check for influencial value
sm.graphics.influence_plot(model2)
#here 48 is looking most influenceable value 

In [ ]:
#dropiing most influencer row
final_startup=final_startup.drop(final_startup.index[48])

In [ ]:
#checking for VIF for checking any feature is invovling in multicollinearaity or not

rsq_st0 = smf.ols('state0~state1+state2+rd+admin+market',data=final_startup).fit().rsquared  
vif_st0 = 1/(1 - rsq_st0) 

rsq_st1 = smf.ols('state1~state0+state2+rd+admin+market',data=final_startup).fit().rsquared  
vif_st1 = 1/(1 - rsq_st1)

rsq_st2 = smf.ols('state2~state0+state1+rd+admin+market',data=final_startup).fit().rsquared  
vif_st2 = 1/(1 - rsq_st2) 

rsq_rd = smf.ols('rd~state0+state1+state2+admin+market',data=final_startup).fit().rsquared  
vif_rd = 1/(1 - rsq_rd) 

rsq_admin = smf.ols('admin~state0+state1+state2+rd+market',data=final_startup).fit().rsquared  
vif_admin = 1/(1 - rsq_admin)

rsq_market = smf.ols('market~state0+state1+state2+rd+admin',data=final_startup).fit().rsquared  
vif_market = 1/(1 - rsq_market) 


d1 = {'Variables':['State0','State1','State02','rd', 'admin', 'market'], 'VIF':[vif_st0,vif_st1,vif_st2,vif_rd, vif_admin, vif_market]}
Vif_frame = pd.DataFrame(d1)  
Vif_frame
# we are getting vif inf for all state column so removing all state feature will help us to  build good model

In [ ]:
#removing State columns as vif is inf
final_startup=final_startup.drop(['state0','state1','state2'],axis=1)

#splitting data in train and test 

from sklearn.model_selection import train_test_split

train,test= train_test_split(final_startup, test_size = 0.10,random_state=10)

new_model=smf.ols('profit~rd+admin+market',data=train).fit()
new_model.summary()

test_pred = new_model.predict(test)
test_pred

test_resid = test_pred - test.profit
# RMSE value for test data 
test_rmse = np.sqrt(np.mean(test_resid * test_resid))
test_rmse

#here test rmse i 5659.455555555s    which is very large  , hence we need more data for geeting better results